In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"qasmovdyant","key":"4d4759066dca189587bcac8981b34740"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ishantjuyal/emotions-in-text

Dataset URL: https://www.kaggle.com/datasets/ishantjuyal/emotions-in-text
License(s): copyright-authors


In [ ]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py

--2025-04-30 06:29:09--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-04-30 06:29:09 (24.0 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data

In [ ]:
unzip_data('/content/emotions-in-text.zip')

In [ ]:
data = pd.read_csv('/content/Emotion_final.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21459 entries, 0 to 21458
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     21459 non-null  object
 1   Emotion  21459 non-null  object
dtypes: object(2)
memory usage: 335.4+ KB


In [ ]:
data.describe()

,Text,Emotion
count,21459,21459
unique,21405,6
top,i was so stubborn and that it took you getting...,happy
freq,2,7029


In [ ]:
df_shuffled = data.sample(frac=1, random_state=42)
df_shuffled

,Text,Emotion
20248,"There was amusement in Jake 's voice , but the...",happy
8094,i feel it is too dangerous to invest in such m...,anger
2859,i feel that i should write the company and tel...,sadness
18953,i also feel ashamed at the hurt caused and ash...,sadness
3931,i just yearned for that homey feeling where yo...,love
...,...,...
11284,i want to savor this feeling of ecstatic antic...,happy
11964,im feeling all puppy dogs and rainbows when im...,sadness
5390,i feel is very delicate,love
860,i was starting to feel a little stressed,sadness


In [ ]:
X = df_shuffled['Text']
y = df_shuffled['Emotion']

In [ ]:
y.value_counts()

,count
Emotion,
happy,7029
sadness,6265
anger,2993
fear,2652
love,1641
surprise,879


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse_output=False)

y_train = onehot.fit_transform(y_train.values.reshape(-1, 1))
y_test = onehot.transform(y_test.values.reshape(-1, 1))

In [ ]:
X_test.head()

,Text
17832,i disagree with my parents on many issues and ...
17612,i needed some space i needed to grow i was in ...
12070,i feel like that nurse reluctant to know where...
12409,i must say that the initial splash was not too...
8379,i feel agitated i become easily overwhelmed


In [ ]:
max_length = round(sum([len(i.split()) for i in X_train]) / (len(X_train)))

In [ ]:
max_length

19

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 10_000
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length = max_length)

In [ ]:
text_vectorizer.adapt(X_train)

In [ ]:
sample_sentence = "He lost his grandma"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 19), dtype=int64, numpy=
array([[  42,  283,   70, 7253,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])>

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length,
                             name = 'embedding')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#Model 0

In [ ]:
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, activation='relu', kernel_regularizer = 'l2')(x)
outputs = layers.Dense(len(data['Emotion'].value_counts()),activation='sigmoid')(x)
model_0 = tf.keras.Model(inputs, outputs, name = 'model_0')

In [ ]:
model_0.compile(loss = 'categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model_0.summary()

Model: "model_0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 19)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 19, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,798 (5.07 MB)

 Trainable params: 1,329,798 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_0_history = model_0.fit(X_train.to_numpy(),
                              y_train,
                              epochs=5,
                              validation_data = (X_test.to_numpy(), y_test))

Epoch 1/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 34s 53ms/step - accuracy: 0.3190 - loss: 1.9498 - val_accuracy: 0.5019 - val_loss: 1.2197
Epoch 2/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - accuracy: 0.5480 - loss: 1.1477 - val_accuracy: 0.6132 - val_loss: 1.0283
Epoch 3/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - accuracy: 0.6511 - loss: 0.8741 - val_accuracy: 0.6626 - val_loss: 1.0487
Epoch 4/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 19s 35ms/step - accuracy: 0.7409 - loss: 0.7430 - val_accuracy: 0.6980 - val_loss: 1.0945
Epoch 5/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 18s 30ms/step - accuracy: 0.7954 - loss: 0.6266 - val_accuracy: 0.7300 - val_loss: 1.0154


In [ ]:
prediction_position = np.argmax(model_0.predict(tf.constant(['She lost her mum'])))
prediction_position

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


np.int64(4)

In [ ]:
models_prediction = onehot.get_feature_names_out()[prediction_position][3:]
models_prediction

'sadness'

In [ ]:
import tensorflow as tf
import numpy as np

#LSTM BLOKU

In [ ]:
def lstm_block():
  ht_prev = np.random.randn(224, 224, 3)
  xt = np.random.randn(224, 224, 3)
  concat = tf.keras.layers.Concatenate()([ht_prev, xt])

  # rt
  rt_W = np.random.randn(224, 224, 6)
  rt_in = rt_W * concat
  rt = tf.keras.layers.Activation('sigmoid')(rt_in)
  rt_out = ht_prev * rt[:, :, :3]

  # zt
  zt_W = np.random.randn(224, 224, 6)
  zt_in = zt_W * concat
  zt = tf.keras.layers.Activation('sigmoid')(zt_in)

  # ht
  ht_W = np.random.randn(224, 224, 6)
  ht_in = ht_W * concat
  ht = tf.keras.layers.Activation('tanh')(ht_in)

  ht_out = zt * rt

  # 1 - zt
  zt_out = ht_prev * (1 - zt)[:, :, :3]

  ht_prev = ht_out[:, :, :3]
  return ht_prev

In [ ]:
for _ in range(20):
  t = lstm_block()

print(t)

tf.Tensor(
[[[0.10749437 0.2809373  0.4195385 ]
  [0.10140277 0.20777513 0.24891254]
  [0.3458063  0.3123541  0.32442704]
  ...
  [0.26596504 0.5140305  0.5843921 ]
  [0.2936668  0.00818859 0.1235032 ]
  [0.00115343 0.04848678 0.14502476]]

 [[0.41014132 0.3083812  0.2465398 ]
  [0.7634639  0.24776201 0.22680655]
  [0.25009128 0.18267736 0.00227731]
  ...
  [0.12121325 0.26305827 0.27682334]
  [0.462425   0.0580699  0.20740324]
  [0.4505812  0.04005548 0.25097305]]

 [[0.28419438 0.23553252 0.21525414]
  [0.18174203 0.31595227 0.05490723]
  [0.2795839  0.4310159  0.5553608 ]
  ...
  [0.31504443 0.04838521 0.32862782]
  [0.3289348  0.00830591 0.3583526 ]
  [0.20495172 0.25327167 0.54263157]]

 ...

 [[0.26569268 0.16699761 0.21990977]
  [0.2362759  0.2822946  0.07933345]
  [0.2529546  0.33487678 0.26361415]
  ...
  [0.3527616  0.24094756 0.24473053]
  [0.14270036 0.1936765  0.07399964]
  [0.72206664 0.27908614 0.16716233]]

 [[0.20162612 0.36524615 0.2582432 ]
  [0.32036015 0.09728266 0